In [1]:
import pandas as pd

In [2]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

C:\Users\Gcarl\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gcarl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import gensim.corpora as corpora
import gensim
from pprint import pprint

## Import datset

In [4]:
URL = "https://raw.githubusercontent.com/ProfAI/natural-language-processing/main/datasets/Lezione_7-Topic_modeling/"

In [5]:
dataset = pd.read_csv(URL+"dataset_Research_Article.csv")
dataset

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [6]:
titles = dataset['TITLE'] + dataset['ABSTRACT']

### Data cleaning function

In [7]:
def sent_to_words(items):
    for item in items:
        yield(gensim.utils.simple_preprocess(str(item), deacc=True))

def remove_stopwords(texts):
    return [[word for word in words if word not in stop_words and len(word) >=5] for words in texts]

## Data Cleaning

In [8]:
data_words = list(sent_to_words(titles))
data_words = remove_stopwords(data_words)

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Vectorize text
corpus = [id2word.doc2bow(text) for text in data_words]

## Model training - Latent Dirichlet Allocation (LDA)

In [10]:
num_topics = 8

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       passes = 3)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"model" + 0.010*"algorithm" + 0.007*"problem" + 0.007*"based" + '
  '0.006*"paper" + 0.005*"using" + 0.005*"optimal" + 0.005*"approach" + '
  '0.005*"method" + 0.005*"algorithms"'),
 (1,
  '0.008*"problem" + 0.006*"equations" + 0.006*"space" + 0.006*"prove" + '
  '0.005*"equation" + 0.005*"solutions" + 0.005*"group" + 0.005*"paper" + '
  '0.005*"results" + 0.005*"mathbb"'),
 (2,
  '0.007*"function" + 0.007*"mathcal" + 0.006*"method" + 0.006*"mathbb" + '
  '0.006*"results" + 0.006*"paper" + 0.005*"using" + 0.005*"functions" + '
  '0.005*"order" + 0.004*"alpha"'),
 (3,
  '0.008*"model" + 0.006*"networks" + 0.006*"models" + 0.005*"learning" + '
  '0.005*"using" + 0.004*"paper" + 0.004*"network" + 0.004*"problem" + '
  '0.004*"based" + 0.004*"number"'),
 (4,
  '0.006*"systems" + 0.005*"study" + 0.005*"software" + 0.005*"properties" + '
  '0.004*"theory" + 0.004*"structure" + 0.004*"analysis" + 0.004*"using" + '
  '0.004*"order" + 0.004*"model"'),
 (5,
  '0.006*"model" + 0.006

In [11]:
unseen_document1 = "NETHIC: A system for automatic text classification using neural networks and hierarchical taxonomies. This paper presents NETHIC, a software system for the automatic classification of textual documents based on hierarchical taxonomies and artificial neural networks. This approach combines the advantages of highly-structured hierarchies of textual labels with the versatility and scalability of neural networks, thus bringing about a textual classifier that displays high levels of performance in terms of both effectiveness and efficiency. The system has first been tested as a general-purpose classifier on a generic document corpus, and then applied to the specific domain tackled by DANTE, a European project that is meant to address criminal and terrorist-related online contents, showing consistent results across both application domains."
title = id2word.doc2bow(simple_preprocess(unseen_document1))
for index, score in lda_model[title]:
    print("TOPIC: "+str(index))
    print("SCORE:"+str(score))

TOPIC: 4
SCORE:0.026298689
TOPIC: 6
SCORE:0.9632412


In [12]:
unseen_document2 = "An Automatic Text Classification Method Based on Hierarchical Taxonomies, Neural Networks and Document Embedding: The NETHIC Tool. This work describes an automatic text classification method implemented in a software tool called NETHIC, which takes advantage of the inner capabilities of highly-scalable neural networks combined with the expressiveness of hierarchical taxonomies. As such, NETHIC succeeds in bringing about a mechanism for text classification that proves to be significantly effective as well as efficient. The tool had undergone an experimentation process against both a generic and a domain-specific corpus, outputting promising results. On the basis of this experimentation, NETHIC has been now further refined and extended by adding a document embedding mechanism, which has shown improvements in terms of performance on the individual networks and on the whole hierarchical model."
title = id2word.doc2bow(simple_preprocess(unseen_document2))
for index, score in lda_model[title]:
    print("TOPIC: "+str(index))
    print("SCORE:"+str(score))

TOPIC: 6
SCORE:0.98635286
